In [1]:
import pandas as pd
import numpy as np
from scipy.spatial.distance import euclidean
import pickle

In [2]:
# File reading

estaciones_aire = pd.read_csv('C:/Users/Mikel/Desktop/Archivos/Estudios/Ciencia de Datos/TFM/Tercera iteración/datos_aire/estaciones_aire.csv')
estaciones_trafico = pd.read_csv('C:/Users/Mikel/Desktop/estaciones_chamberi.csv')
estaciones_trafico = estaciones_trafico[['id','utm_x', 'utm_y']]
air = pd.read_csv('C:/Users/Mikel/Desktop/Archivos/Estudios/Ciencia de Datos/TFM/Tercera iteración/datos_aire/121.csv', sep = ';')

In [3]:
def nearest_station():
    '''We check which air quality station is the nearest to each car traffic station'''
    ids = list(estaciones_trafico['id'])
    codigos = list(estaciones_aire['CODIGO_CORTO'])
    aire_x = estaciones_aire['COORDENADA_X_ETRS89']
    aire_y = estaciones_aire['COORDENADA_Y_ETRS89']
    aire_coords = list(zip(aire_x, aire_y))
    trafico_x = estaciones_trafico['utm_x']
    trafico_y = estaciones_trafico['utm_y']
    trafico_coords = list(zip(trafico_x, trafico_y))
    ans_dict = {}
    dist_dict = {}
    # We measure euclidean distances with several nested loops
    for i in range(0, len(trafico_coords)):
        for c in range(0, len(aire_coords)):
            dist = euclidean(aire_coords[c], trafico_coords[i])
            if ids[i] not in dist_dict.keys():
                dist_dict[ids[i]] = (dist, codigos[c])
            if dist < dist_dict[ids[i]][0]:
                dist_dict[ids[i]] = (dist, codigos[c])
    nearest_list = []
    for key, value in dist_dict.items():
        ans_dict[key] = value[1]
    return ans_dict

In [4]:
# We apply the function

nearest = nearest_station()
estaciones_trafico['nearest'] = nearest
unique_stations = set(nearest.values())

# We take a look at the stations that are included in the nearest stations
print(nearest)

{4417: 4, 3807: 4, 3846: 35, 4413: 35, 4412: 35, 4411: 4, 3407: 48, 3748: 48, 4440: 48, 3848: 35, 3847: 48, 4430: 48, 4450: 48, 4446: 48, 4428: 48, 4386: 4, 4389: 4, 4388: 4, 10605: 4, 4422: 4, 10023: 4, 4423: 4, 10057: 4, 10022: 4, 10217: 4, 4384: 4, 5130: 4, 4385: 4, 10604: 4, 4380: 4, 3747: 4, 10499: 4, 10498: 4, 3850: 8, 3851: 8, 4432: 48, 4431: 48, 3674: 48, 4449: 48, 4448: 48, 3562: 48, 4429: 48, 4447: 48, 4438: 48, 4452: 48, 4426: 48, 4451: 48, 4436: 48, 4442: 48, 4461: 48, 4398: 4, 4397: 4, 7067: 4, 4383: 4, 4410: 4, 4418: 4, 4407: 4, 4416: 4, 4408: 4, 4406: 4, 4414: 48, 4415: 48, 4404: 38, 4403: 38, 4409: 38, 4433: 38, 4421: 38, 3693: 38, 7018: 38, 4456: 38, 4427: 48, 4445: 48, 3622: 48, 4458: 48, 4459: 48, 4457: 48, 9965: 48, 9967: 48, 9963: 48, 9964: 48, 4405: 48, 4434: 48, 4453: 48, 4439: 48, 4441: 48, 4464: 38, 4455: 38, 5728: 38, 5712: 38, 7140: 38, 4454: 38, 5696: 38, 5710: 38, 5739: 48, 5714: 48, 5740: 48, 5713: 38, 5716: 48, 5741: 38, 5738: 38, 5717: 38, 5711: 38, 4469

In [5]:
def air_quality_reading(year):
    '''Function to read the air quality file'''
    base_path = 'C:/Users/Mikel/Desktop/Archivos/Estudios/Ciencia de Datos/TFM/Tercera iteración/datos_aire/'
    return_df = pd.DataFrame()
    if year == 2021:
        file_marker = 121
        ran_ge = 12
    elif year == 2022:
        file_marker = 122
        ran_ge = 10
    for i in range(ran_ge):
        file_path = base_path + str(file_marker) +'.csv'
        concat_df = pd.read_csv(file_path, sep = ';')
        return_df = pd.concat([return_df, concat_df])
        file_marker += 100
    return(return_df)

def air_quality_preprocessing(air, start, end, year):
    '''Function to get the relevant info from the air quality file'''
    return_dict = {}
    air_subset = air[(air["ESTACION"] == 4) | (air["ESTACION"] == 8) | (air["ESTACION"] == 35) \
                 | (air["ESTACION"] == 38) | (air["ESTACION"] == 48)]
    air_subset = air_subset.drop(columns=['PROVINCIA', 'MUNICIPIO', 'PUNTO_MUESTREO'])
    air_subset_magns = air_subset[(air_subset["MAGNITUD"] == 7) | (air_subset["MAGNITUD"] == 8) \
                             | (air_subset["MAGNITUD"] == 12)]
    stations = [4, 8, 35, 38, 48]
    magns = [7, 8, 12]
    if year == 2021:
        z = 13
    if year == 2022:
        z = 11
    months = list(range(1, z))
    h_v = []
    for i in range(1, 25):
        if i < 10:
            h = 'H0' + str(i)
            v = 'V0' + str(i)
        else:
            h = 'H' + str(i)
            v = 'V' + str(i)
        h_v.append((h, v))
    # Transposing of the dataframe
    for station in stations:
        air_subset_station = air_subset_magns[(air_subset_magns["ESTACION"] == station)]
        station_df = pd.DataFrame()
        for magn in magns:
            air_subset_station_magns = air_subset_station[(air_subset_station["MAGNITUD"] == magn)]
            magn_list = []
            station_months = air_subset_station_magns['MES'].unique().tolist()
            for month in months:
                air_subset_month = air_subset_station_magns[(air_subset_station_magns["MES"] == month)]
                day_list = air_subset_month['DIA'].unique().tolist()
                if month in [1, 3, 5, 7, 8, 10, 12]:
                    days_month = list(range(1, 32))
                elif month in [4, 6, 9, 11]:
                    days_month = list(range(1, 31))
                elif month == 2:
                    days_month = list(range(1, 29))
                if month not in station_months:
                    for item in days_month:
                        for j in range(24):
                            magn_list.append(0)
                    continue
                last_day = days_month[-1]
                for day in days_month:
                    if day not in day_list:
                        for k in range(24):
                            magn_list.append(0)
                    elif day in day_list:
                        air_subset_day = air_subset_month[(air_subset_month["DIA"] == day)]
                        for item in h_v:
                            if not air_subset_day.empty: 
                                h = air_subset_day.iloc[0][item[0]]
                                v = air_subset_day.iloc[0][item[1]]
                                if v == 'V':
                                    magn_list.append(h)
                                elif v != 'V':
                                    magn_list.append(0)
            station_df[magn] = magn_list
        dater = pd.date_range(start = start, end = end, freq='H', closed='left')
        station_df = station_df.set_index(dater)
        return_dict[station] = station_df
    return return_dict

In [6]:
# We read and preprocess the air quality files

air_2021 = air_quality_reading(2021)
air_2022 = air_quality_reading(2022)
air_dict_2021 = air_quality_preprocessing(air_2021, '1/1/2021', '1/1/2022', 2021)
air_dict_2022 = air_quality_preprocessing(air_2022, '1/1/2022', '11/1/2022', 2022)

In [7]:
# We organize the items as a dict

air_dict = {}
for key, df in air_dict_2021.items():
    air_dict[key] = pd.concat([df, air_dict_2022[key]])

In [8]:
# We subset the desired stations

estaciones_aire = estaciones_aire[(estaciones_aire["CODIGO_CORTO"] == 4) | (estaciones_aire["CODIGO_CORTO"] == 8) \
                                  | (estaciones_aire["CODIGO_CORTO"] == 35) | (estaciones_aire["CODIGO_CORTO"] == 38) \
                                  | (estaciones_aire["CODIGO_CORTO"] == 48)]

In [9]:
# We create a dict with the nearest stations to each traffic station

stations = [4, 8, 35, 38, 48]
nearest_dist = {}
nearest = {}
x = list(estaciones_aire['COORDENADA_X_ETRS89'])
y = list(estaciones_aire['COORDENADA_Y_ETRS89'])
coords = list(zip(x, y))
for i in range(0, len(coords)):
    for j in range(0, len(coords)):
        dist = euclidean(coords[i], coords[j])
        if dist == 0:
            continue
        elif stations[i] not in nearest.keys():
            nearest_dist[stations[i]] = dist
            nearest[stations[i]] = stations[j] 
        elif dist < nearest[stations[i]]:
            nearest_dist[stations[i]] = dist
            nearest[stations[i]] = stations[j] 

In [10]:
# If a station has data missing, we will use another station's data

air_dict_corrected = {}
for key, df in air_dict.items():
    corrected_df = pd.DataFrame()
    sub_station = nearest[key]
    corrected_df[7] = np.where((df[7] == 0), air_dict[sub_station][7], df[7])
    corrected_df[8] = np.where((df[8] == 0), air_dict[sub_station][8], df[8])
    corrected_df[12] = np.where((df[12] == 0), air_dict[sub_station][12], df[12])
    corrected_df = corrected_df.set_index(df.index)
    air_dict_corrected[key] = corrected_df

In [11]:
# We create the final dict

air_dict_final = {}
for key, df in air_dict_corrected.items():
    df = df.asfreq(freq='15min')
    df = df.interpolate(method='time')
    air_dict_final[key] = df

In [12]:
# Exporting pickle

with open('C:/Users/Mikel/Desktop/Archivos/Estudios/Ciencia de Datos/TFM/Tercera iteración/air_quality_dict.pickle' \
          , 'wb') as handle:
    pickle.dump(air_dict_final, handle, protocol=pickle.HIGHEST_PROTOCOL)
    
with open('C:/Users/Mikel/Desktop/Archivos/Estudios/Ciencia de Datos/TFM/Tercera iteración/nearest_dict.pickle' \
          , 'wb') as handle:
    pickle.dump(nearest, handle, protocol=pickle.HIGHEST_PROTOCOL)